# Clew Based Genetic Algorithm

An example of how we could implement a 'Clew Based' genetric algorithm with easily swapable cost and crossover functions.

### General Setup

General Setup that can be shared between the different runs

In [4]:
# Various imports
import random
import matplotlib
import numpy as np
from src.algorithms.basic_ga import BasicGeneticAlgorithm
from src.image_loading import load_image
from src.image_manipulation import crop
from src.worm import CamoWorm, Clew
from src.worm_mask import WormMask
from src.helpers import clamp
from src import rng
from src.progress_image_generator import ProgressImageGenerator, build_gif

# Controls the size of the plots.
matplotlib.rcParams['figure.dpi'] = 140

# Various Constants
POPULATION_SIZE = 400
CLEW_SIZE = 400
GENERATIONS_PER_ITERATION = 10

# Load the image
image = crop(load_image("images", "original"), (320, 560, 160, 880))

def random_clew_function() -> Clew:
    return [CamoWorm.random(image.shape) for _ in range(CLEW_SIZE)]

def iterate_and_print(algorithm: BasicGeneticAlgorithm) -> None:
    progress_image_generator = ProgressImageGenerator(image, "Basic Genetic", "./progress")

    for index in range(100):
        results = algorithm.run_generations(GENERATIONS_PER_ITERATION)

        print(
            f"({results[-1].generation:04}) {results[-1].costs[0]:.2f} {results[-1].duration*GENERATIONS_PER_ITERATION}")

        clew = algorithm.population[0].underlying
        worm_masks = [WormMask(worm, image) for worm in clew]
        progress_image_generator.save_progress_image(clew, worm_masks, index * GENERATIONS_PER_ITERATION)
    
    build_gif("./progress", "progress.gif")

### Crossover functions

In [2]:
def fully_random_crossover(clew1: Clew, clew2: Clew) -> Clew:
    """ Worms essentially get randomly shuffled and can end up at any index. """
    cross_index = rng.integers(0, CLEW_SIZE)

    new_clew = random.choices(
        clew1, k=cross_index) + random.choices(clew2, k=(CLEW_SIZE-cross_index))

    return new_clew


def mutate_worm(worm: CamoWorm) -> CamoWorm:
    """ Mutates the given worm. """
    (ylim, xlim) = image.shape

    return CamoWorm(
        worm.x,
        worm.y,
        worm.r + rng.standard_normal() * 0.1,
        worm.theta + np.pi * 0.1 * rng.standard_normal(),
        worm.dr + rng.standard_normal() * 0.1,
        worm.dgamma + np.pi * 0.1 * rng.standard_normal(),
        max(worm.width + rng.standard_normal() * 0.1, 0),
        clamp(worm.colour + rng.standard_normal() * 0.1, 0, 1)
    )


def index_respecting_crossover(clew1: Clew, clew2: Clew) -> Clew:
    """ Worms will always remain at the same index and cannot change position
    in the clew. """
    cross_index = rng.integers(0, CLEW_SIZE)

    if random.random() > 0.5:
        new_clew = clew1[:cross_index] + clew2[cross_index:]
    else:
        new_clew = clew2[:cross_index] + clew1[cross_index:]

    for index, worm in enumerate(new_clew):
        if random.random() < 0.05:
            new_clew[index] = mutate_worm(worm)

    return new_clew


### Cost functions

In [6]:
def max_width_cost_function(clew: Clew) -> float:
    cost = 0.0

    for worm in clew:
        cost += 1 / worm.width

    return cost

def min_width_cost_function(clew: Clew) -> float:
    cost = 0.0

    for worm in clew:
        cost += worm.width

    return cost

def min_theta_cost_function(clew: Clew) -> float:
    cost = 0.0

    for worm in clew:
        cost += abs(worm.theta)

    return cost

### Max Width

Max width using the fully random crossover function

In [ ]:
max_width_random_crossover = BasicGeneticAlgorithm(
    POPULATION_SIZE, max_width_cost_function, random_clew_function, fully_random_crossover)

iterate_and_print(max_width_random_crossover)

Using the index respecting crossover

In [ ]:
max_width_index_respecting = BasicGeneticAlgorithm(
    POPULATION_SIZE, max_width_cost_function, random_clew_function, index_respecting_crossover)

iterate_and_print(max_width_index_respecting)

### Min Width

In [ ]:
min_width_random_crossover = BasicGeneticAlgorithm(
    POPULATION_SIZE, min_width_cost_function, random_clew_function, fully_random_crossover)

iterate_and_print(min_width_random_crossover)

In [ ]:
min_width_index_respecting = BasicGeneticAlgorithm(
    POPULATION_SIZE, min_theta_cost_function, random_clew_function, index_respecting_crossover)

iterate_and_print(min_width_index_respecting)